# <code>geo_bezier_3d</code>
------------------------------------
### Para dúvidas maiores and/or lista de exemplos, veja a [documentação](https://geo-bezier-3d.readthedocs.io/).

In [ ]:
lx,ly,lz=8,2,2

nx,ny,nz=256,129,129

lenght_position=0.25*lx

wide_centralize_term=((lz/2)-(1.15/2))

raf=True

ramp_angle=25 #degrees

In [ ]:
dx,dy,dz=lx/(nx-1),ly/(ny-1),lz/(nz-1)

In [ ]:
name = str(input('Give your project a name: '))

In [ ]:
open(f'inputs.py', 'w').close()
with open(f'inputs.py', 'a') as the_file:
    the_file.write(f'lx={lx}\n'), the_file.write(f'ly={ly}\n'), the_file.write(f'lz={lz}\n')
    the_file.write(f'nx={nx}\n'), the_file.write(f'ny={ny}\n'), the_file.write(f'nz={nz}\n')  
    the_file.write(f'dx={dx}\n'), the_file.write(f'dy={dy}\n'), the_file.write(f'dz={dz}\n')  
    the_file.write('name='),the_file.write(f'"'), the_file.write(f'{name}'), the_file.write(f'"')        

In [ ]:
from infos import *
from inputs import *
from creating_solid import *
import inputs as i
import infos as s
import creating_solid as c

In [ ]:
gen_raf_information(nraf=3)

In [ ]:
#cleaning any mess

c.eq_storage={}
c.list_storage={}

#creating solids for front fillet representation

#we're going to create only half ahmeds body. after all process, we'll include a mirror function.

gen_cylinder('1','down horz','xy',radius=0.295,
                                  center_1=0.295+lenght_position,
                                  center_2=0.443,
                                  init_height=0.00+wide_centralize_term, 
                                  final_height=0.575+wide_centralize_term)

gen_cylinder('2','up horz','xy',radius=0.295,
                                center_1=0.295+lenght_position,
                                center_2=0.704,
                                init_height=0.00+wide_centralize_term,
                                final_height=0.575+wide_centralize_term)

gen_cylinder('3','right vert','xz',radius=0.295,
                                   center_1=0.295+lenght_position,
                                   center_2=0.295+wide_centralize_term,
                                   init_height=0.148,
                                   final_height=1.00)

#intersection's collect first round

gen_epsi_cylinder('1', 'solid', add_or_sub='add', cyl_raf_path=raf)
gen_epsi_cylinder('3', 'solid', add_or_sub='add', cyl_raf_path=raf)

normalize_epsi(intersection=True,target=2,epsi_raf_path=raf)

#intersection's collect second round

gen_epsi_cylinder('2', 'solid', add_or_sub='add', cyl_raf_path=raf)
gen_epsi_cylinder('3', 'solid', add_or_sub='add', cyl_raf_path=raf)

normalize_epsi(intersection=True,target=2,epsi_raf_path=raf)

#another solids to front fillet representation

gen_cylinder('4','down hor','xy',radius=0.295,
                                 center_1=0.295+lenght_position,
                                 center_2=0.443,
                                 init_height=0.295+wide_centralize_term, 
                                 final_height=0.575+wide_centralize_term)

gen_cylinder('5','up hor','xy',radius=0.295,
                               center_1=0.295+lenght_position,
                               center_2=0.704,
                               init_height=0.295+wide_centralize_term,
                               final_height=0.575+wide_centralize_term)

gen_cylinder('6','right vert','xz',radius=0.295,
                                   center_1=0.295+lenght_position,
                                   center_2=0.295+wide_centralize_term,
                                   init_height=0.443,
                                   final_height=0.704)

#validating those solids to epsi

gen_epsi_cylinder('4', 'solid', cyl_raf_path=raf)
gen_epsi_cylinder('5', 'solid', cyl_raf_path=raf)
gen_epsi_cylinder('6', 'solid', cyl_raf_path=raf)

#filling the middle hole

gen_quad_prism('7',a=0.295*2,b=0.704-0.443,c=0.575-0.295,reference_point=[lenght_position,0.443,0.295+wide_centralize_term])

gen_epsi_quad_prism('7','solid',qp_raf_path=raf)

#creating ahmeds body

gen_quad_prism('8',a=3.088-0.295,b=1.000-0.147,c=0.575,reference_point=[0.295+lenght_position,0.147,wide_centralize_term,])

gen_epsi_quad_prism('8','solid',qp_raf_path=raf)

#creating back ramp dimensions

hipo=0.656

adj=math.cos(math.radians(ramp_angle))*hipo
opo=math.sin(math.radians(ramp_angle))*hipo

#transforming those dimensions to a bezier surface

set_point_matrix(2,2)

point_storage['P00']=[(3.088-adj),1.00,0.000]
point_storage['P01']=[(3.088-adj),1.00,0.575]

point_storage['P10']=[3.088,1.00-opo,0.000]
point_storage['P11']=[3.088,1.00-opo,0.575]

create_point_matrix(True)

translate('x',lenght_position) #term to shift ahmed in domain - lenght
translate('z',wide_centralize_term) #term to centralize ahmed in domain - wide

gen_bezier_surface('9','ramp back')

gen_epsi_bezier_surface('entry+exit and/or exit','zy','9', add_or_sub='sub', bez_raf_path=raf)

#creating 'wheels'

gen_cylinder('10','right f wheel','xz',radius=0.082/2,
                                       center_1=0.591+lenght_position,
                                       center_2=0.137+wide_centralize_term,
                                       init_height=0.00, 
                                       final_height=0.147)

gen_cylinder('11','right b wheel','xz',radius=0.082/2,
                                       center_1=2.145+lenght_position,
                                       center_2=0.137+wide_centralize_term,
                                       init_height=0.00,
                                       final_height=0.147)

gen_epsi_cylinder('10', 'solid', cyl_raf_path=raf)
gen_epsi_cylinder('11', 'solid', cyl_raf_path=raf)

#mirror the whole domain

gen_epsi_mirror('whole_domain','z', mirror_raf_path=raf)

#get epsi matrix

gen_output(name,out_raf_path=raf)